# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import json
import geoviews as gv
import cartopy as crs
import pyproj
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aitape,-3.1374,142.3475,78.71,88,100,8.97,PG,PG
1,1,ushuaia,-54.8000,-68.3000,60.46,41,75,14.97,AR,AR
2,2,mahibadhoo,3.7833,72.9667,80.51,76,12,6.80,MV,MV
3,3,san patricio,28.0170,-97.5169,63.86,89,100,4.56,US,US
4,4,khatanga,71.9667,102.5000,16.50,96,100,7.02,RU,RU


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [58]:
# Configure the map plot
humidity = city_data_df["Humidity"].astype(float)
locations = city_data_df[["Lat", "Lng"]]

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 1,
    size = "Humidity",
    color = "City"
)

# Display the map plot_1
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values

# set the ideal temperature limits
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 65)\
#set wind speed under 5 mph #set cloudiness at 0
& (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather_df.dropna()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,kapaa,22.0752,-159.3190,70.12,77,0,8.05,US,US
37,37,pisco,-13.7000,-76.2167,69.85,78,0,8.05,PE,PE
55,55,makakilo city,21.3469,-158.0858,69.98,82,0,4.61,US,US
156,156,camana,-16.6228,-72.7111,73.74,61,0,9.78,PE,PE
186,186,hilo,19.7297,-155.0900,78.28,93,0,5.75,US,US
238,238,constitucion,-35.3333,-72.4167,66.78,49,0,5.30,CL,CL
314,314,puerto escondido,15.8500,-97.0667,79.63,78,0,4.61,MX,MX
328,328,bera,24.0833,89.6167,70.97,54,0,6.49,BD,BD
403,403,chabahar,25.2919,60.6430,78.33,58,0,7.52,IR,IR
465,465,iquique,-20.2208,-70.1431,71.64,66,0,7.11,CL,CL


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(ideal_weather_df, columns=["City", "Country", "Lat", "Lng"]).copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,kapaa,US,22.0752,-159.3190,
37,pisco,PE,-13.7000,-76.2167,
55,makakilo city,US,21.3469,-158.0858,
156,camana,PE,-16.6228,-72.7111,
186,hilo,US,19.7297,-155.0900,
238,constitucion,CL,-35.3333,-72.4167,
314,puerto escondido,MX,15.8500,-97.0667,
328,bera,BD,24.0833,89.6167,
403,chabahar,IR,25.2919,60.6430,
465,iquique,CL,-20.2208,-70.1431,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "categories": "accomodation.hotel",
        "key": geoapify_key,
        "limit": 20
        }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    response = response.json()
       
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



Starting hotel search
kapaa - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
camana - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
constitucion - nearest hotel: No hotel found
puerto escondido - nearest hotel: No hotel found
bera - nearest hotel: No hotel found
chabahar - nearest hotel: No hotel found
iquique - nearest hotel: No hotel found
tanda - nearest hotel: No hotel found
dolores - nearest hotel: No hotel found


In [62]:
# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,kapaa,US,22.0752,-159.3190,No hotel found
37,pisco,PE,-13.7000,-76.2167,No hotel found
55,makakilo city,US,21.3469,-158.0858,No hotel found
156,camana,PE,-16.6228,-72.7111,No hotel found
186,hilo,US,19.7297,-155.0900,No hotel found
238,constitucion,CL,-35.3333,-72.4167,No hotel found
314,puerto escondido,MX,15.8500,-97.0667,No hotel found
328,bera,BD,24.0833,89.6167,No hotel found
403,chabahar,IR,25.2919,60.6430,No hotel found
465,iquique,CL,-20.2208,-70.1431,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [55]:
# Configure the map plot
hotels = hotel_df["Hotel Name"]
locations = hotel_df[["Lat", "Lng"]]

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 1,
    size = "accomodation.hotel",
    color = "Country")

    
# Display the map

map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country)